In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from ast import literal_eval
from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader

import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

In [ ]:
train = pd.read_csv('/kaggle/input/wsdm-kkbox/train.csv')
test = pd.read_csv('/kaggle/input/wsdm-kkbox/test.csv')
songs = pd.read_csv('/kaggle/input/wsdm-kkbox/songs.csv')
members = pd.read_csv('/kaggle/input/wsdm-kkbox/members.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/wsdm-kkbox/train.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                  'target' : np.uint8,
                                                  'song_id' : 'category'})

In [ ]:
test = pd.read_csv('/kaggle/input/wsdm-kkbox/test.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                'source_screen_name' : 'category',
                                                'source_type' : 'category',
                                                'song_id' : 'category'})

In [ ]:
songs = pd.read_csv('/kaggle/input/wsdm-kkbox/songs.csv',dtype={'genre_ids': 'category',
                                                  'language' : 'category',
                                                  'artist_name' : 'category',
                                                  'composer' : 'category',
                                                  'lyricist' : 'category',
                                                  'song_id' : 'category'})

In [ ]:
songs.head(5)

In [ ]:
members = pd.read_csv('/kaggle/input/wsdm-kkbox/members.csv',dtype={'city' : 'category',
                                                      'bd' : np.uint8,
                                                      'gender' : 'category',
                                                      'registered_via' : 'category'},parse_dates=['registration_init_time','expiration_date'])

In [ ]:
songs_extra = pd.read_csv('/kaggle/input/wsdm-kkbox/song_extra_info.csv')

In [ ]:
songs_extra

In [ ]:
#song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs, on='song_id', how='left')
test = test.merge(songs, on='song_id', how='left')
train = train.merge(songs_extra, on='song_id', how='left')
test = test.merge(songs_extra, on='song_id', how='left')
del songs_extra

In [ ]:
train_members = pd.merge(train, members, on='msno', how='left')


In [ ]:
1+1

In [ ]:
test_members = pd.merge(test, members, on='msno', how='left')

In [ ]:
train_members.head(5)

In [ ]:

train_members_songs = pd.merge(train_members, songs, on='song_id', how='left')
test_members_songs = pd.merge(test_members, songs, on='song_id', how='left')

In [ ]:
train_members_songs.to_csv('train_members_songs.csv')
test_members_songs.to_csv('test_members_songs.csv')

In [ ]:
train_members_songs.info()

In [ ]:
#del train_members_songs
#del test_members_songs
del train_members
del test_members
#del members


In [ ]:
members.head(5)

In [ ]:
train_members_songs.head(5)

In [ ]:
train_members_songs['registration_year'] = train_members_songs['registration_init_time'].apply(lambda x: int(str(x)[0:4]))


In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

np.set_printoptions(threshold=np.inf)

In [ ]:
train_members_songs.head(1)

In [ ]:
members.head(1)

In [ ]:
train_members_songs['expiration_year'] = train_members_songs['expiration_date'].apply(lambda x: int(str(x)[0:4]))
train_members_songs['expiration_month'] = train_members_songs['expiration_date'].apply(lambda x: int(str(x)[4:6]))

In [ ]:

train_members_songs['membership_days'] = (train_members_songs['expiration_date'] - train_members_songs['registration_init_time']).dt.days.astype(int)

In [ ]:
# categorize membership_days 
train_members_songs['membership_days'] = train_members_songs['membership_days']//200
train_members_songs['membership_days'] = train_members_songs['membership_days'].astype('category')

In [ ]:
train_members_songs.info()

In [ ]:
# member_cols = ['msno','city','registered_via', 'registration_year', 'expiration_year', 'membership_days']

# train = train.merge(members[member_cols], on='msno', how='left')
# test = test.merge(members[member_cols], on='msno', how='left')

In [ ]:

def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return int(isrc[5:7])//5
        else:
            return int(isrc[5:7])//5
    else:
        return np.nan
#categorize song_year per 5years
#song year could be a feature

train_members_songs['song_year'] = train_members_songs['isrc'].apply(isrc_to_year)
test_members_songs['song_year'] = test_members_songs['isrc'].apply(isrc_to_year)
#songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

In [ ]:
1+1

In [ ]:
train_members_songs['genre_ids'] = train_members_songs['genre_ids_x'].str.split('|').str[0]

In [ ]:
temp_song_length = train_members_songs['song_length_x']
temp_song_length

In [ ]:
train_members_songs = pd.read_csv('/kaggle/working/train_members_songs.csv')
test_members_songs = pd.read_csv('/kaggle/working/test_members_songs.csv')

In [ ]:
# train.drop('song_length', axis = 1, inplace = True)
# test.drop('song_length',axis = 1 , inplace =True)

members.head(1)

In [ ]:
#delete useless column
train_members_songs.drop('genre_ids_y',axis=1, inplace=True)
test_members_songs.drop('genre_ids_y',axis=1, inplace=True)

In [ ]:
song_count = train_members_songs.loc[:,["song_id","target"]]
# measure repeat count by played songs
song_count1 = song_count.groupby(["song_id"],as_index=False).sum().rename(columns={"target":"repeat_count"})
# count play count by songs
song_count2 = song_count.groupby(["song_id"],as_index=False).count().rename(columns = {"target":"play_count"})

In [ ]:
song_repeat = song_count1.merge(song_count2,how="inner",on="song_id")
song_repeat["repeat_percentage"] = round((song_repeat['repeat_count']*100) / song_repeat['play_count'],1)
song_repeat['repeat_count'] = song_repeat['repeat_count'].astype('int')
song_repeat['repeat_percentage'] = song_repeat['repeat_percentage'].replace(100.0,np.nan)
#cuz most of 100.0 are played=1 repeated=1 values. I think it is not fair compare with other played a lot songs

In [ ]:
train_members_songs = train_members_songs.merge(song_repeat,on="song_id",how="left")
test_members_songs = test_members_songs.merge(song_repeat,on="song_id",how="left")

In [ ]:
del song_repeat

In [ ]:
train_members_songs.head(5)

In [ ]:
# type cast
train_members_songs['song_id'] = train_members_songs['song_id'].astype('category')
train_members_songs['repeat_count'] = train_members_songs['repeat_count'].fillna(0)
train_members_songs['repeat_count'] = train_members_songs['repeat_count'].astype('int')
train_members_songs['play_count'] = train_members_songs['play_count'].fillna(0)
train_members_songs['play_count'] = train_members_songs['play_count'].astype('int')

test_members_songs['song_id'] = test_members_songs['song_id'].astype('category')
test_members_songs['repeat_count'] = test_members_songs['repeat_count'].fillna(0)
test_members_songs['repeat_count'] = test_members_songs['repeat_count'].astype('int')
test_members_songs['play_count'] = test_members_songs['play_count'].fillna(0)
test_members_songs['play_count'] = test_members_songs['play_count'].astype('int')

In [ ]:

artist_count = train_members_songs.loc[:,["artist_name_x","target"]]

num = train_members_songs.isna().sum()

# measure repeat count by played songs
artist_count1 = artist_count.groupby(["artist_name_x"],as_index=False).sum().rename(columns={"target":"repeat_count_artist"})

# measure play count by songs
artist_count2 = artist_count.groupby(["artist_name_x"],as_index=False).count().rename(columns = {"target":"play_count_artist"})

artist_repeat = artist_count1.merge(artist_count2,how="inner",on="artist_name_x")

In [ ]:
# measure repeat count by played songs
artist_count1 = artist_count.groupby(["artist_name_x"],as_index=False).sum().rename(columns={"target":"repeat_count_artist"})

# measure play count by songs
artist_count2 = artist_count.groupby(["artist_name_x"],as_index=False).count().rename(columns = {"target":"play_count_artist"})

artist_repeat = artist_count1.merge(artist_count2,how="inner",on="artist_name_x")

In [ ]:

artist_count = train_members_songs.loc[:,["artist_name_x","target"]]
# measure repeat count by played songs
artist_count1 = artist_count.groupby(["artist_name_x"],as_index=False).sum().rename(columns={"target":"repeat_count_artist"})
# measure play count by songs
artist_count2 = artist_count.groupby(["artist_name_x"],as_index=False).count().rename(columns = {"target":"play_count_artist"})
artist_repeat = artist_count1.merge(artist_count2,how="inner",on="artist_name_x")

In [ ]:
artist_repeat.shape

In [ ]:
del artist_repeat['repeat_count_artist']

In [ ]:
train_members_songs = train_members_songs.merge(artist_repeat,on="artist_name_x",how="left")


In [ ]:
test_members_songs = test_members_songs.merge(artist_repeat,on="artist_name_x",how="left")

In [ ]:
del artist_repeat

In [ ]:
msno_count = train_members_songs.loc[:,["msno","target"]]

# count repeat count by played songs
msno_count1 = msno_count.groupby(["msno"],as_index=False).sum().rename(columns={"target":"repeat_count_msno"})

# count play count by songs
msno_count2 = msno_count.groupby(["msno"],as_index=False).count().rename(columns = {"target":"play_count_msno"})

msno_repeat = msno_count1.merge(msno_count2,how="inner",on="msno")

In [ ]:
msno_repeat["repeat_percentage_msno"] = round((msno_repeat['repeat_count_msno']*100) / msno_repeat['play_count_msno'],1)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].fillna(0)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].astype('int')
#msno_repeat['repeat_percentage_msno'] = msno_repeat['repeat_percentage_msno'].replace(100.0,np.nan)
# it can be meaningful so do not erase 100.0

In [ ]:
train_members_songs = train_members_songs.merge(msno_repeat,on="msno",how="left")
test_members_songs = test_members_songs.merge(msno_repeat,on="msno",how="left")

In [ ]:
for col in train_members_songs.columns:
    if train_members_songs[col].dtype == object:
        train_members_songs[col] = train_members_songs[col].astype('category')
        test_members_songs[col] = test_members_songs[col].astype('category')

In [ ]:
train_members_songs.head(1)

In [ ]:
train_members_songs['song_year'] = train_members_songs['song_year'].astype('category')
test_members_songs['song_year'] = test_members_songs['song_year'].astype('category')


In [ ]:
train_members_songs['play_count_msno'] = train_members_songs['play_count_msno'].fillna(0)
test_members_songs['play_count_msno'] = test_members_songs['play_count_msno'].fillna(0)


train_members_songs['play_count_artist'] = train_members_songs['play_count_artist'].fillna(0)
test_members_songs['play_count_artist'] = test_members_songs['play_count_artist'].fillna(0)

train_members_songs['play_count_artist'] = train_members_songs['play_count_artist'].astype('int')
test_members_songs['play_count_artist'] = test_members_songs['play_count_artist'].astype('int')

train_members_songs['play_count_msno'] = train_members_songs['play_count_msno'].astype('int')
test_members_songs['play_count_msno'] = test_members_songs['play_count_msno'].astype('int')



In [ ]:
train_members_songs.head(1)

In [ ]:
!rm '/kaggle/working/train_members_songs.csv'
!rm '/kaggle/working/test_members_songs.csv'

In [ ]:
!ls '/kaggle/input/'

In [ ]:
train_members_songs.to_csv('train_features.csv')

In [ ]:
test_members_songs.to_csv('test_features.csv')

In [ ]:
train_features = pd.read_csv('/kaggle/working/train_features.csv')
test_features = pd.read_csv('/kaggle/working/test_features.csv')

In [ ]:
del train_members_songs
del test_members_songs

In [ ]:
train_features.shape

In [ ]:
train_song_ids = train_features['song_id']
test_song_ids = test_features['song_id']

In [ ]:
from pandas import Series,DataFrame
df = DataFrame(train_song_ids)
df2 = DataFrame(test_song_ids)

In [ ]:
df['song_int_id'] = range(len(df))
df2['song_int_id'] = range(len(df2))

In [ ]:
df.head(5)

In [ ]:
df2.head(5)

In [ ]:
train_features = train_features.merge(df, on='song_id', how='inner')
test_features = test_features.merge(df2, on='song_id', how='inner')